In [2]:
import csv

def import_csv(csvfile):
    """
    Imports the CSV file as an array of lines
    
    :param csvfile: The path of the CSV file
    :return lines: The lines of the CSV file 
    """
    lines = []
    with open(log_path) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)

    lines.pop(0) # Remove the header info from the CSV file
    return lines

In [3]:
data_path = './data/'
img_path = data_path + 'IMG/'
log_path = data_path + 'driving_log.csv'

# Store the lines of the CSV file in an array
lines = []
lines = import_csv(log_path)

In [4]:
# Test to make sure we read that properly
print(lines[0])

['/Users/greg/Documents/Udacity/CarND/term1/CarND-Behavioral-Cloning-P3/data/IMG/center_2017_05_28_10_49_49_163.jpg', '/Users/greg/Documents/Udacity/CarND/term1/CarND-Behavioral-Cloning-P3/data/IMG/left_2017_05_28_10_49_49_163.jpg', '/Users/greg/Documents/Udacity/CarND/term1/CarND-Behavioral-Cloning-P3/data/IMG/right_2017_05_28_10_49_49_163.jpg', '-0.04981203', '0', '0', '5.219624']


In [5]:
# Transform to a relative path to the images
for line in lines:
    line[0] = line[0].split('/')[-1]
    line[1] = line[1].split('/')[-1]
    line[2] = line[2].split('/')[-1]
    
# Test
print(lines[0])

['center_2017_05_28_10_49_49_163.jpg', 'left_2017_05_28_10_49_49_163.jpg', 'right_2017_05_28_10_49_49_163.jpg', '-0.04981203', '0', '0', '5.219624']


In [6]:
import cv2
import numpy as np

left_images = []
center_images = []
right_images = []
meas = []

# Make lines a numpy array
lines = np.array(lines)

# Import the images and the corresponding steering measurements
center_images = lines[:,0]
left_images = lines[:,1]
right_images = lines[:,2]
steer_meas = lines[:,3]

# Test to make sure we got that right
print(left_images[0:5])
print(center_images[0:5])
print(right_images[0:5])
print(steer_meas[0:5])

print(len(left_images))
print(len(center_images))
print(len(right_images))
print(len(steer_meas))

['left_2017_05_28_10_49_49_163.jpg' 'left_2017_05_28_10_49_49_231.jpg'
 'left_2017_05_28_10_49_49_309.jpg' 'left_2017_05_28_10_49_49_382.jpg'
 'left_2017_05_28_10_49_49_460.jpg']
['center_2017_05_28_10_49_49_163.jpg' 'center_2017_05_28_10_49_49_231.jpg'
 'center_2017_05_28_10_49_49_309.jpg' 'center_2017_05_28_10_49_49_382.jpg'
 'center_2017_05_28_10_49_49_460.jpg']
['right_2017_05_28_10_49_49_163.jpg' 'right_2017_05_28_10_49_49_231.jpg'
 'right_2017_05_28_10_49_49_309.jpg' 'right_2017_05_28_10_49_49_382.jpg'
 'right_2017_05_28_10_49_49_460.jpg']
['-0.04981203' '-0.04981203' '-0.04981203' '-0.04981203' '-0.04981203']
45714
45714
45714
45714


In [7]:
# Transform the measurements array to float
steer_meas = steer_meas.astype(np.float32)

# Test the new data type
print(steer_meas[0:5])

[-0.04981203 -0.04981203 -0.04981203 -0.04981203 -0.04981203]


In [8]:
# Apply an offset to left and right steering angles
steer_offset = 0.25
left_steer_meas = steer_meas + steer_offset
right_steer_meas = steer_meas - steer_offset

# Test the steering offset
print(left_steer_meas[0:5])
print(right_steer_meas[0:5])

[ 0.20018797  0.20018797  0.20018797  0.20018797  0.20018797]
[-0.29981202 -0.29981202 -0.29981202 -0.29981202 -0.29981202]


In [9]:
# Put the paths for left, right, and center images into one dataset
samples = []
samples.extend(center_images)
samples.extend(left_images)
samples.extend(right_images)

# Test that
print(len(samples))
print(samples[0], samples[46000], samples[95000])

137142
center_2017_05_28_10_49_49_163.jpg left_2017_05_28_10_50_10_314.jpg right_2017_05_28_10_56_42_360.jpg


In [10]:
# Put the angles for left, right, and center images into one dataset
meas = []
meas.extend(steer_meas)
meas.extend(left_steer_meas)
meas.extend(right_steer_meas)

# Test that
print(len(meas))
print(meas[0], meas[46000], meas[95000])

137142
-0.049812 0.186413 -0.273687


In [11]:
from sklearn.utils import shuffle

# Shuffle the datasets in unison
samples, meas = shuffle(samples, meas)

# Test the output
print(samples[0:10])
print(meas[0:10])

['right_2017_05_28_12_35_23_385.jpg', 'right_2017_05_28_11_08_40_211.jpg', 'right_2017_05_28_11_16_00_814.jpg', 'center_2017_05_28_11_23_37_513.jpg', 'right_2017_05_28_12_14_00_310.jpg', 'center_2017_05_28_12_40_01_854.jpg', 'right_2017_05_28_11_56_46_807.jpg', 'center_2017_05_28_11_38_45_509.jpg', 'right_2017_05_28_12_26_51_014.jpg', 'center_2017_05_28_11_09_37_254.jpg']
[-0.26512569, -0.30327773, -0.4596599, 0.094557673, -0.1325188, -0.029619951, -0.29847568, 0.1170407, -0.18164062, 0.002643327]


In [12]:
from sklearn.model_selection import train_test_split

# Separate the data into training and validation sets
# Note: samples=X_, meas=y_
X_train, X_valid, y_train, y_valid = train_test_split(samples, meas, test_size=0.25, random_state=42)

# Make sure we did that properly
print(X_train[0:5])
print(y_train[0:5])
print(X_valid[0:5])
print(y_valid[0:5])

['center_2017_05_28_11_33_03_326.jpg', 'right_2017_05_28_11_02_06_556.jpg', 'center_2017_05_28_12_25_20_084.jpg', 'right_2017_05_28_11_57_15_755.jpg', 'center_2017_05_28_12_25_55_728.jpg']
[-0.099286303, -0.2643474, 0.026492011, -0.3799783, 0.18960001]
['left_2017_05_28_11_21_37_921.jpg', 'center_2017_05_28_11_16_01_562.jpg', 'left_2017_05_28_12_14_46_440.jpg', 'left_2016_12_01_13_45_19_077.jpg', 'left_2017_05_28_12_14_02_878.jpg']
[0.25738662, -0.1709352, 0.26530194, 0.10479359, 0.2889744]


In [13]:
# Define the shape of the trimmed image to run the model on
ch, rw, col = 160, 320, 3

In [14]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Dropout

def nvidia_model(): 
    """
    NVIDIA model
    """
    model = Sequential()
    model.add(Cropping2D(cropping=((70, 25), (0, 0)),
                         input_shape=(ch, rw, col)))
    model.add(Lambda(lambda x: x / 255.0 - 0.5))
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation="relu"))
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation="relu"))
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation="relu"))
    model.add(Convolution2D(64, 3, 3, activation="relu"))
    model.add(Convolution2D(64, 3, 3, activation="relu"))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))

    model.compile(loss='mse', optimizer='adam')
    return model

Using TensorFlow backend.


In [15]:
import cv2

def process_img(image):
    """
    Preprocessing for an image
    
    :param image: The image to be processed
    :return img: The processed image
    """
    # Convert to YUV color space
    img = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    return img

def generator(samples, measurements, batch_size=32):
    """
    Generator to reduce memory footprint when training
    
    :param samples: The array of images to process
    :param measurements: The array of steering angle measurements
    :param batch_size: The number of samples to process concurrently
    :yield: The generator
    """
    assert len(samples) == len(measurements)
    num_samples = len(samples)

    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_meas = measurements[offset:offset+batch_size]

            images = []
            angles = []
            for i, sample_val in enumerate(batch_samples):
                image = cv2.imread(img_path + sample_val, cv2.IMREAD_COLOR)
                angle = batch_meas[i]
                img = process_img(image)

                # Flip horizontally
                img = cv2.flip(img, 0)
                angle = -angle

                images.append(img)
                angles.append(angle)

            X_arr = np.array(images)
            y_arr = np.array(angles)
            yield shuffle(X_arr, y_arr)

In [16]:
def plot_results(history):
    """
    Plot the results 
    
    :param history: The fit model
    """
    # Print the keys in the history object
    print(history.history.keys())

    # Plot training and validation loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.savefig("training_validation_loss_plot.jpg")
    plt.show()

In [17]:
# Compile and train the model using the generator
batch_size = 128
num_epochs = 3

train_generator = generator(X_train, y_train, batch_size)
valid_generator = generator(X_valid, y_valid, batch_size)

model = nvidia_model()
model.save('model.h0')

fit_model = model.fit_generator(train_generator,
                                samples_per_epoch=3*len(X_train),
                                validation_data=valid_generator,
                                nb_val_samples=len(X_valid),
                                nb_epoch=num_epochs,
                                verbose=1)

plot_results(fit_model)

Epoch 1/3
308568/308568 [==============================] - 3061s - loss: 0.0196 - val_loss: 0.0129

NameError: name 'plt' is not defined